In [90]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [91]:
adata = read_h5ad('/Users/madelinepark/Downloads/Limb_Muscle_facs.h5ad')
tiss = adata

In [142]:
def loc_split(tiss, feature):
    tiss.obs['feature_type_of_interest'] = 'rest'
    results_feature_cv = pd.DataFrame()
    for c in list(set(tiss.obs[feature])): 
        print(c)
        clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
        selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration  
        feature_of_interest = c
        tiss.obs.loc[tiss.obs[tiss.obs[feature] == feature_of_interest].index,'feature_type_of_interest'] = feature_of_interest
        feat_labels = tiss.var_names 
        X = tiss.X
        y = tiss.obs['feature_type_of_interest']
#     return clf, selector, X, y, feat_labels, feature
    return X, y, feature, feat_labels

In [143]:
def make_forest (n_estimators, random_state, oob_score, step, cv, n_jobs):
    clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration
    return clf, selector

In [151]:
def train(X, y, test_size, random_state):
    print('training...')
    feat_labels = loc_split(tiss=tiss, feature='age')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 
    clf, selector = make_forest (n_estimators=10, random_state=0, oob_score=True, step=0.2, cv=3, n_jobs=-1)
    clf.fit(X_train, y_train)
    selector.fit(X_train, y_train)
    feature_selected = feat_labels[selector.support_]
    return feat_labels, feature_selected, selector.estimator_.feature_importances_, X_train, X_test, y_train, y_test

In [152]:
def result_write (c, feature_selected, tiss, feature_type_of_interest): #feature_selected comes from above function... hm
    print('result writing')
    feat_labels = train(X, y, test_size, random_state) #or put as loc_split from above 
    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected
    resaux[c + '_gini'] = (selector.estimator_.feature_importances_)
    print(feature_selected)
    print (selector.estimator_.feature_importances_)
    results_feature_cv = pd.concat([results_feature_cv,resaux],axis=1)
    tiss.obs['feature_type_of_interest'] = 'rest' #this line is to reset the loop, but is not specific to this specific sub function, just didn't know where to put

In [153]:
# def scRFE (tiss=tiss, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5) :
#     for c in list(set(tiss.obs[feature])): 
#         clf, selector, X, y = loc_split(tiss, feature)
#         make_forest(n_estimators=10, random_state=0, oob_score=True, step=0.2, cv=2, n_jobs=-1)
#         feature_selected, selector.estimator_.feature_importances_, X_train, X_test, y_train, y_test 
#         train(X, y, test_size, random_state)
#         result_write (c, feature_selected, tiss, feature_type_of_interest)

In [154]:
def scRFE (tiss=tiss, feature='age', n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5) :
    for c in list(set(tiss.obs[feature])): 
        X, y, feature, feat_labels = loc_split(tiss=tiss, feature= 'age')
        clf, selector = make_forest(n_estimators=10, random_state=0, oob_score=True, step=0.2, cv=2, n_jobs=-1)
        feature_selected, selector.estimator_.feature_importances_, X_train, X_test, y_train, y_test = train(X, y, test_size, random_state)
        result_write (c, feature_selected, tiss, feature_type_of_interest)

In [155]:
scRFE(tiss=tiss, feature='age', n_estimators=10, random_state=0, n_jobs=-1, oob_score=True, test_size = 0.05, step=0.2, cv=5)

3m
24m
training...
3m
24m


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

TypeError: only integer scalar arrays can be converted to a scalar index

## its angry bc clf is only defined within the feature importance method... 
# so how do I fix this